# Anime Recommender System

## I. Problem Definition

Salah satu hal yang menarik pada bentuk recommender system di bidang entertainment yaitu anime adalah bagaimana kita melakukan atau memberikan rekomendasi untuk menaikkan/meretain user engagement. 

Entertainment dalam bentuk layanan streaming sudah bukan lagi hal yang baru/mewah bagi sebagian penonton. Konten layanan streaming ini bervariase dari layanan video, musik, film, hingga anime. Berbagai macam layanan streaming (OTT) anime pun sudah banyak bermunculan seperti cruchyroll, netflix, hulu, bstation, dll sehingga persaingan yang cukup ketat pun tidak akan terelakkan.

Pilihan banyaknya anime baik judul maupun genre banyak tersedia dalam layanan OTT tersebut. Namun perlu diingat bahwa user memiliki keterbatasan waktu untuk mencari anime yang cocok dengan seleranya pada saat ingin menontonnya. Hal ini dapat menjadi kesempatan bagi OTT untuk dapat memberikan rekomendasi anime yang setepat mungkin dengan selera penonton. Salah satu metode/ solusi yang dapat digunakan adalah menggunakan perhitungan recommender system untuk mengetahui atau meberikan rekomendasi yang tepat kepada penonton.

Pilihan rekomendasi yang tepat tersebut dapat memberikan kenyamanan dan kemudahan bagi user utnuk menonton anime sehingga dapat meningkatkan engagement pada layanan OTT untuk tetap berlangganan dan pada ujungnya dapat meretain atau bahkan meningkatkan revenue perusahaan OTT.

## II. Data Gathering and Preparation

Data yang diambil berasal dari situs Myanimelist.com yang terdiri dari user yang memberikan rating terhadap suatu anime. Data tersebut dibagi menjadi 2 data yaitu :
1. Data anime, yang memuat database mengenai Anime yang telah berada pada situs Myanimelist.com
2. Data rating, yang memuat database user yang telah memberikan rating pada Data Anime

In [1]:
# Load this library
import numpy as np
import pandas as pd

In [2]:
# Import Data User yang memberikan rating pada anime
rating = pd.read_csv(r'D:\PELATIHAN DATA SCIENCE\recommender\rating.csv')
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


Karena keterbatasan source hardware yang digunakan, maka data yang akan diambil adalah sebanyak 20% dari seluruh data rating

In [3]:
# Ambil 20% data rating
rating = rating.sample(frac=0.20,replace=False)

In [4]:
# Tampilkan jumlah data
print(f'rating shape: {rating.shape}')

rating shape: (1562747, 3)


In [5]:
# check Duplicate pada data rating
dup_rating = rating[rating.duplicated()].shape[0]
print(f'count of duplicated rating: {dup_rating}')

count of duplicated rating: 0


In [6]:
#remove duplicate
rating.drop_duplicates(keep='first',inplace=True)

dup_rating = rating[rating.duplicated()].shape[0]
print(f'count of duplicated rating after removing: {dup_rating}')

count of duplicated rating after removing: 0


In [7]:
#jumlah setelah check duplicate
print(f'rating shape: {rating.shape}')
print('tidak terdapat duplicate pada data rating')

rating shape: (1562747, 3)
tidak terdapat duplicate pada data rating


In [8]:
# Check Isian rating
print(rating["rating"].unique())

[ 6  5  8  9  7 -1 10  3  4  2  1]


Isian rating terdapat angka "-1" yang artinya bahwa user menonton anime tersebut namun tidak memberikan rating. Oleh sebab itu data tersebut dapat dihilangkan sehingga menghasilkan data net sebagai berikut:

In [9]:
#hilangkan rating -1
rating = rating[rating["rating"] >= 0]

In [10]:
#jumlah setelah menghilangkan -1
print(f'rating shape: {rating.shape}')

rating shape: (1267791, 3)


## III. Data Splitting

Dalam mempermudah proses kalkulasi untuk data splitting hingga modelling kita gunakan package Surprise, nantinya package terebut membuat matrix Utility serta membagi data menjadi Full data, Trainset, dan Testset.

In [11]:
# Import some library
from surprise import Dataset, Reader

In [12]:
# Buat reader
reader = Reader(rating_scale = (1, 10))
reader

In [13]:
# Buat Matriks Utility data
utility_data = Dataset.load_from_df(
                    df = rating[['anime_id','user_id', 'rating']].copy(),
                    reader = reader
                )

utility_data

In [14]:
# Tampilkan Utility data
utility_data.df.head()

,anime_id,user_id,rating
2134661,527,20655,6
5667557,19315,53176,5
1292572,1575,12244,8
307810,6347,3155,8
3418474,16668,31513,9


Selanjutnya kita buat fungsi untuk membagi data menjadi Traiset dan Testset.

In [15]:
# Create a function to split data
def train_test_split(utility_data, test_size, random_state):
    """
    Train test split the data
    ref: https://surprise.readthedocs.io/en/stable/FAQ.html#split-data-for-unbiased-estimation-py

    Parameters
    ----------
    utility_data : Surprise utility data
        The sample of whole data set

    test_size : float, default=0.2 (membagi datatest sebanyaj 20%)
        The test size

    random_state : int, default=42 (hasil akan berbeda tergantung parameter randomstate yang digunakan)
        For reproducibility

    Returns
    -------
    full_data : Surprise utility data
        The new utility data

    train_data : Surprise format
        The train data

    test_data : Surprise format
        The test data
    """
    # Deep copy the utility_data
    full_data = copy.deepcopy(utility_data)

    # Generate random seed
    np.random.seed(random_state)

    # Shuffle the raw_ratings for reproducibility
    raw_ratings = full_data.raw_ratings
    np.random.shuffle(raw_ratings)

    # Define the threshold
    threshold = int((1-test_size) * len(raw_ratings))

    # Split the data
    train_raw_ratings = raw_ratings[:threshold]
    test_raw_ratings = raw_ratings[threshold:]

    # Get the data
    full_data.raw_ratings = train_raw_ratings
    train_data = full_data.build_full_trainset()
    test_data = full_data.construct_testset(test_raw_ratings)

    return full_data, train_data, test_data

In [16]:
# Load library for deep copy
import copy
# Split the data dengan porsi 20% data test dan random state 42
full_data, train_data, test_data = train_test_split(utility_data,
                                                    test_size = 0.2,
                                                    random_state = 42)

In [17]:
# Validate the splitting
print(f' Jumlah data train sebanyak {train_data.n_ratings}')
print(f' Jumlah data test sebanyak {len(test_data)}')

 Jumlah data train sebanyak 1014232
 Jumlah data test sebanyak 253559


## IV. Data Preprocessing

In [18]:
# Check Missing Value
rating.isna().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [19]:
print("missing value yang butuh mendapat perhatian adalah user rating,anime id, dan user id tidak terdapat missing data ")

missing value yang butuh mendapat perhatian adalah user rating,anime id, dan user id tidak terdapat missing data 


## V. Modelling

In [20]:
! pip install surprise
import surprise

Defaulting to user installation because normal site-packages is not writeable


#### Baseline Model
Model yang akan kita gunakan adalah model Baseline dengan metode Mean Prediction

In [21]:
# Import Library
from surprise.model_selection.search import RandomizedSearchCV
from surprise import AlgoBase, KNNBaseline, SVD

In [22]:
class MeanPrediction(AlgoBase):
    '''Baseline prediction. Return global mean as prediction'''
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        '''Fit the train data'''
        AlgoBase.fit(self, trainset)

    def estimate(self, u, i):
        '''Perform the estimation/prediction.'''
        est = self.trainset.global_mean
        return est

In [23]:
# Creating baseline model instance
model_baseline = MeanPrediction()
model_baseline

In [24]:
# Import the cross validation module
from surprise.model_selection import cross_validate

In [25]:
# Use full_data for cross validation
# Your results could be different because
# there is no random seed stated within this functions
cv_baseline = cross_validate(algo = model_baseline,
                             data = full_data,
                             cv = 5,
                             measures = ['rmse'])

In [26]:
# Extract CV eith RMSE results
cv_baseline_rmse = cv_baseline['test_rmse'].mean()
print(f' Nilai RMSE untuk Basemodel adalah {cv_baseline_rmse}')

 Nilai RMSE untuk Basemodel adalah 1.5715454060613


#### k-Nearest Neighbor kNN

Kita akan merekomendasikan anime berdasarkan rating maka approach dan task yang tepat digunakan adalah regresi, sehingga kNN dapat digunakan sebagai recommender system dengan menggunakan rating item lain sebagai referensi dalam merekomendasi (Item Based) Collaborative Filtering dan penggunaan pearson similarity yang lebih efektif daripada cosine (jurnal telkom University).

In [27]:
# Set parameter menggunakan perhitungan similarity pearson_baseline dan menggunakan User Based CF
sim_options = {
    'name': 'pearson_baseline',
    'user_based': 'False'
}

In [28]:
knn_baseline = KNNBaseline(sim_options = sim_options)

In [29]:
knn_baseline.fit(train_data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [30]:
# Use full_data for cross validation
# Your results could be different because
# there is no random seed stated within this functions
cv_knn = cross_validate(algo = knn_baseline,
                             data = full_data,
                             cv = 5,
                             measures = ['rmse'])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [31]:
knn_baseline_rmse = cv_knn['test_rmse'].mean()
print(f' Nilai RMSE untuk kNNBaseline adalah {knn_baseline_rmse}')

 Nilai RMSE untuk kNNBaseline adalah 1.288986957052117


#### Singular Value Decomposition (SVD)
Selain peggunaan algoritma KNN, collaborative filtering dapat dilakukan menggunakan Matrix Factorization (MF), cara kerjanya yaitu dengan mendekomposisi interaksi matriks item - user menjadi produk dari dua matriks dua dimensi yang lebih rendah. yaitu User Matrix di mana baris mewakili pengguna dan kolom adalah latent factor. Satu lagi yaitu Item Matrix di mana baris merupakan latent factor dan kolom mewakili item.

In [32]:
# Creating baseline model instance
svd = SVD()

In [33]:
svd.fit(train_data)

In [34]:
# Use full_data for cross validation
# Your results could be different because
# there is no random seed stated within this functions
cv_svd = cross_validate(algo = svd,
                             data = full_data,
                             cv = 5,
                             measures = ['rmse'])

In [35]:
svd_rmse = cv_svd['test_rmse'].mean()
print(f' Nilai RMSE untuk SVD adalah {svd_rmse}')

 Nilai RMSE untuk SVD adalah 1.2814901784770778


#### Performance Comparisons
Bandingkan Nilai RMSE antara ketiga model yang telah di buat.

In [36]:
# Membuat Tabel Summary
summary_df = pd.DataFrame({'Model': ['Baseline', 'kNN-Baseline', 'SVD'],
                           'CV Performance - RMSE': [cv_baseline_rmse,knn_baseline_rmse,svd_rmse ],
                           })

summary_df

,Model,CV Performance - RMSE
0,Baseline,1.571545
1,kNN-Baseline,1.288987
2,SVD,1.281490


Model yang cukup baik adalah model SVD oleh karena itu kita lakukan parameter tuning untuk memperoleh hasil yang optimal.

## VI. SVD Hyperparameter Tuning

Hyperparameter Candidate :
- Learning Rate (𝛾) terdiri dari beberapa nilai : [0.5,0.05,0.005]
- Jumlah Latent Factor beberapa nilai : [50,200]
- Regularization Strength beberapa nilai :  [0.2,0.01,0.02]

In [37]:
# Set parameter SVD
params_SVD = {'lr_all' : [0.5,0.05,0.005], 'n_factors' : [50,200],
              'reg_all' : [0.2,0.01,0.02]
              }

In [38]:
# Buat Tuning SVD
tuning_svd = RandomizedSearchCV(algo_class=SVD, param_distributions = params_SVD,
                   cv=5
                   )

In [39]:
# Lakukan Tuning SVD pada full data
tuning_svd.fit(data=full_data)

In [40]:
# Best Score SVD
tuning_svd_rmse = tuning_svd.best_score['rmse']

# Best Param SVD
best_params_svd = tuning_svd.best_params['rmse']

### Evaluate Model

In [41]:
summary_df = pd.DataFrame({'Model': ['SVD', 'SVD-Tuned'],
                           'CV Performance - RMSE': [svd_rmse,tuning_svd_rmse ],
                           'Model Configuration':['N/A',f'{best_params_svd}']})
summary_df

,Model,CV Performance - RMSE,Model Configuration
0,SVD,1.281490,N/A
1,SVD-Tuned,1.242318,"{'lr_all': 0.05, 'n_factors': 200, 'reg_all': ..."


Setelah mendapatkan best parameter yang terbukti menghasilkan RMSE lebih baik, selanjutnya adalah retrain data tersebut dengan menggunakan best parameter.

#### Retrain data with best parameter

In [42]:
# Retrain SVD dengan Best Parameter
# Create object
model_best = SVD(**best_params_svd)

# Retrain on whole train dataset
model_best.fit(train_data)

#### Hitung Akurasi data dengan menggunakan Data Test

In [43]:
# import performance library
from surprise import accuracy

In [44]:
# Hitung Prediksi
test_pred = model_best.test(test_data)
test_rmse = accuracy.rmse(test_pred)
test_rmse

RMSE: 1.2347


1.2347260606386212

In [45]:
# Summary RMSE
summary_test_df = pd.DataFrame({'Model' : ['SVD'],
                                'RMSE-Tuning': [tuning_svd.best_score['rmse']],
                                'RMSE-Test': [test_rmse]})

summary_test_df

,Model,RMSE-Tuning,RMSE-Test
0,SVD,1.242318,1.234726


Berdasarkan RMSE tersebut maka model menghasilkan prediksi test yang baik sehingga kita dapat menggunakannya untuk memprediksi User.

## VII. Prediction

In [46]:
# Membuat Sample Prediction
sample_prediction = model_best.predict(uid = 9,
                                      iid = 10)

In [47]:
sample_prediction

Prediction(uid=9, iid=10, r_ui=None, est=8.41249640544237, details={'was_impossible': False})

The results tell us

- r_ui : actual rating --> None, means user 9 have yet rated movie 10
- est : the estimated rating from our model
- details : whether prediction is impossible or not. So it's possible to predict.

#### Membuat Fungsi Prediksi

In [48]:
# Membuat Fungsi Prediksi
def get_unrated_item(userid, rating):
    """
    Get unrated item id from a user id

    Parameters
    ----------
    userid : int
        The user id

    rating_data : pandas DataFrame
        The rating data

    Returns
    -------
    unrated_item_id : set
        The unrated item id
    """
    # Find the whole item id
    unique_item_id = set(rating['anime_id'])

    # Find the item id that was rated by user id
    rated_item_id = set(rating.loc[rating['user_id']==userid, 'anime_id'])

    # Find the unrated item id
    unrated_item_id = unique_item_id.difference(rated_item_id)

    return unrated_item_id


In [49]:
# Buat Fungsi Unrated Anime
def get_pred_unrated_item(userid, estimator, unrated_item_id):
    """
    Get the predicted unrated item id from user id

    Parameters
    ----------
    userid : int
        The user id

    estimator : Surprise object
        The estimator

    unrated_item_id : set
        The unrated item id

    Returns
    -------
    pred_data : pandas Dataframe
        The predicted rating of unrated item of user id
    """
    # Initialize dict
    pred_dict = {
        'user_id': userid,
        'anime_id': [],
        'predicted_rating': []
    }

    # Loop for over all unrated movie Id
    for id in unrated_item_id:
        # Create a prediction
        pred_id = estimator.predict(uid = pred_dict['user_id'],
                                    iid = id)

        # Append
        pred_dict['anime_id'].append(id)
        pred_dict['predicted_rating'].append(pred_id.est)

    # Create a dataframe
    pred_data = pd.DataFrame(pred_dict).sort_values('predicted_rating',
                                                     ascending = False)

    return pred_data

#### Load Anime Database
Digunakan untuk memberikan keterangan nama dan genre anime

In [50]:
# Load Anime Database
def load_anime_data(anime_path):
    """
    Load movie data from the given path

    Parameters
    ----------
    anime_path : str
        The movie data path

    Returns
    -------
    anime : pandas DataFrame
        The movie metadata
    """
    # Load data
    anime = pd.read_csv(anime_path,
                             index_col='anime_id',
                             delimiter=',')

    print('Movie data shape :', anime.shape)
    return anime

In [51]:
# Define the anime path
anime_path = r'D:\PELATIHAN DATA SCIENCE\recommender\anime.csv'

In [52]:
# Load anime data using upper function
anime = load_anime_data(anime_path = anime_path)

anime.head()

Movie data shape : (12294, 6)


,name,genre,type,episodes,rating,members
anime_id,,,,,,
32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


#### Search Top Anime
Digunakan untuk mencari anime dengan rating tinggi untuk setiap user

In [53]:
def get_top_highest_unrated(estimator, k, userid, rating, anime):
    """
    Get top k highest of unrated anime from a Surprise estimator RecSys

    Parameters
    ----------
    estimator : Surprise model
        The RecSys model

    k : int
        The number of Recommendations

    userid : int
        The user Id to recommend

    rating : pandas Data Frame
        The rating data

    anime : pandas DataFrame
        The anime meta data

    Returns
    -------
    top_item_pred : pandas DataFrame
        The top items recommendations
    """
    # 1. Get the unrated item id of a user id
    unrated_item_id = get_unrated_item(userid=userid, rating=rating)

    # 2. Create prediction from estimator to all unrated item id
    predicted_unrated_item = get_pred_unrated_item(userid = userid,
                                                   estimator = estimator,
                                                   unrated_item_id = unrated_item_id)

    # 3. Sort & add meta data
    top_item_pred = predicted_unrated_item.head(k).copy()
    top_item_pred['name'] = anime.loc[top_item_pred['anime_id'], 'name'].values
    top_item_pred['genre'] = anime.loc[top_item_pred['anime_id'], 'genre'].values

    return top_item_pred

#### Menampilkan Rekomendasi Anime untuk setiap User

In [54]:
# Generate 5 recommendation for user 100
get_top_highest_unrated(estimator=model_best,
                        k=5,
                        userid=100,
                        rating=rating,
                        anime=anime)

,user_id,anime_id,predicted_rating,name,genre
8277,100,31540,9.773838,Sekkou Boys,"Comedy, Music"
3537,100,4353,9.755825,Gakuen Heaven: Hamu Hamu Heaven,"Comedy, Drama, Romance, School, Shounen Ai"
2921,100,3269,9.743974,.hack//G.U. Trilogy,"Action, Fantasy, Game, Sci-Fi"
6850,100,18689,9.713090,Diamond no Ace,"Comedy, School, Shounen, Sports"
2901,100,3231,9.643351,Gunslinger Girl: Il Teatrino,"Action, Drama, Military, Sci-Fi"
